# 딥러닝 온라인 1:1 과외반 <Level 5> 과제

(이 쥬피터 노트북은 다음의 링크 <b>https://bit.ly/dsd-0502-MX3S</b> 에서 다운받으실 수 있습니다)

지금까지 계산 그래프 (Computational Graph)에 대해 배웠습니다. 
이 Computational Graph를 이용하여 편미분을 사람이 직접 하지않고 프로그램 차원에서 자동화 할 수 있었습니다. 

이번 과제에서는 
**1)** Computational Graph 를 복습하면서 수업시간에 만들어 보지 않았던 **새로운 노드들(로그함수, 제곱(Square)노드, 삼각함수 노드)을 추가로 만들어보고**, 

**2)** 이 Computational Graph의 개념을 활용하여 지금까지 배워왔던 **Loss Function(즉, MSE와 Cross Entropy)에 대한 편미분값을 자동으로 구해주는 코드를 작성**해 볼 것입니다. 

## Computational Graph 복습

computational graph는 계산과정을 그래프로 나타난 것으로, 노드는 연산을 엣지는 입출력 관계를 의미합니다. computational graph를 통해 아무리 복잡한 계산도 각 노드별로의 local gradient만 계산하기 때문에 효율적으로 계산이 가능합니다. 각 노드는 자신과 관련된 계산 외에는 고려하지 않고 중간 미분 결과를 공유하여 다수의 미분을 효율적으로 계산합니다. chain rule의 과정을 시각적으로 표현한 것이라고도 볼 수 있습니다. 

<img src="http://drive.google.com/uc?export=view&id=14x4zQpEEatgMb1W0BY47lXKjM5haZq1x" width="600">


>- **Forward Propagation**

>  **순전파(forward propagation)**는 뉴럴 네트워크의 입력층부터 출력층까지 순서대로 변수들을 계산하고 저장합니다. 그래프 상에서 왼쪽에서 오른쪽으로 진행되는 파란색으로 표시된 계산입니다.
  

>- **Back Propagation**

>  **역전파(back propagaton)**는 네트워크 전체에 대해 반복적으로 chain rule을 적용하여 gradient를 계산하여 뉴럴 네트워크를 효율적으로 학습하는 데에 사용되는 알고리즘입니다. 오차를 역 방향으로 전파하는 방법이라 오차역전파법이라고도 말합니다. 중간 변수와 가중치에 대한 gradient를 계산하고 저장합니다. 그래프 상에서 오른쪽에서 왼쪽으로 진행되는 빨간색으로 표시된 계산입니다.

#### Multiply 노드의 Forward/Back Propagation를 구하는 Class를 아래와 같이 만들어 보았습니다. 

- #### Multiply 노드

곱셈 노드의 수식이 $z=f(x,y)=x\times y $일 때 $z$의 gradient는 아래와 같습니다. 

$$
\nabla z = \left ( \frac{\partial z}{\partial x},\frac{\partial z}{\partial y}  \right ) = \left ( \frac{\partial (xy)}{\partial x},\frac{\partial (xy)}{\partial y}  \right ) = (y,x)
$$

<img src="http://drive.google.com/uc?export=view&id=1pQ9HFmr9_31daD8YIhO72IQtr5Kvj2GP" width="600">


>- **Forward Propagation**

>  곱셈 노드는 입력된 두 가지 값을 곱하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  곱셈 노드는 입력된 값에 forward propagation 당시의 입력 변수들을 서로 바꾸어 곱하여 다음 노드로 전달합니다.

In [1]:
# 곱셈 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class Multiply:
    
    # f(x,y) = xy
    def forward(self, x, y):
        self.x = x
        self.y = y
        # 두 가지 값을 곱하여 다음 노드로 전달합니다.
        return self.x * self.y
    
    # dfdx = y, dfdy = x
    def backward(self):
        dx = self.y
        dy = self.x
        # forward propagation의 입력 변수들을
        # 서로 바꾸어 곱하여 다음 노드로 전달합니다.
        return dx, dy

In [2]:
multiply=Multiply()
forward2 = multiply.forward(10,3)
forward2

30

In [3]:
multiply.backward()

(3, 10)

### 1. 로그함수 노드 구현해보기

자연대수 $e$를 밑으로 하는 로그함수 노드의 수식이 $z=f(x)=log(x)$일 때 $z$의 gradient는 아래와 같습니다.


$$
\nabla z = \left ( \frac{\partial z}{\partial x}\right ) = \left ( \frac{\partial (log(x))}{\partial x}\right ) = \frac{1}{x}
$$

<img src="http://drive.google.com/uc?export=view&id=1YKou4QvGFwjWV_zzk0H8cMS0vGT7lFMB" width="600">

>- **Forward Propagation**

>  로그 노드는 입력된 값을 밑이 $e$인 로그함수의 진수로 취하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  로그 노드는 입력된 값에 역수를 취한 값을 곱하여 다음 노드로 전달합니다.

In [4]:
# 로그를 계산할 수 있는 numpy 패키지를 가져옵니다. np라는 축약어로 사용합니다. 
import numpy as np

In [5]:
# 로그 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class Log:
    
    # f(x) = log(x)
    def forward(self, x):
        self.x = x
        # 로그를 취하여 다음 노드로 전달합니다.
        return np.log(self.x)
    
    # dfdx = 1/x
    def backward(self):
        dx = 1/self.x
        # forward propagation의 입력 변수에 역수를 취한 값을 곱하여 다음 노드로 전달합니다.
        return dx

log 노드 객체를 생성하고, **x=2**값을 넣어준 후 forward, backward값이 각각 **0.693, 0.5** 으로 잘 구해지는지 확인해 봅시다.

In [6]:
# log 노드 객체를 생성합니다.
log=Log()
# x=2 값을 대입해줍니다.
x=2
# x를 입력받았을 때, log노드의 forward, backward 값을 출력해줍니다. 
print(log.forward(x), log.backward())

0.6931471805599453 0.5


### 2. 제곱(Square) 노드 구현해보기

제곱 노드의 수식이 $z=f(x)=x^2$일 때 $z$의 gradient는 아래와 같습니다.


$$
\nabla z = \left ( \frac{\partial z}{\partial x}\right ) = \left ( \frac{\partial x^2}{\partial x}\right ) = 2x
$$

<img src="http://drive.google.com/uc?export=view&id=1C67JqOdnW4dxbLBVHLxF8Hvrr2NoTth3" width="600">


>- **Forward Propagation**

>  로그 노드는 입력된 값에 제곱을 취하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  로그 노드는 입력된 값에 $2x$를 곱하여 다음 노드로 전달합니다.

In [7]:
# 제곱 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class Square:
    
    # f(x) = x^2
    def forward(self, x):
        self.x = x
        # 제곱을 취하여 다음 노드로 전달합니다.
        return self.x * self.x
    
    # dfdx = 2x
    def backward(self):
        dx = 2*self.x
        # 2x를 곱하여 다음 노드로 전달합니다.
        return dx

square 노드 객체를 생성하고, **x=5**값을 넣어준 후 forward, backward값이 각각 **25, 10** 으로 잘 구해지는지 확인해 봅시다.

In [8]:
# square 노드 객체를 생성합니다.
square=Square()
# x=5 값을 대입해줍니다.
x=5
# x를 입력받았을 때, square 노드의 forward, backward 값을 출력해줍니다. 
print(square.forward(x), square.backward())

25 10


### 3. 삼각함수(sin, cos, tan) 노드 구현해보기

#### 3-1. sin함수 노드 구현해보기
sin함수 노드의 수식이 $z=f(x)=sin(x)$일 때 $z$의 gradient는 아래와 같습니다.


$$
\nabla z = \left ( \frac{\partial z}{\partial x}\right ) = \left ( \frac{\partial sin(x)}{\partial x}\right ) = cos(x)
$$
<img src="http://drive.google.com/uc?export=view&id=1zdwScJP5hbMTtJETFVo1P9F8ZACoU-fc" width="600">

>- **Forward Propagation**

>  sin 노드는 입력된 값에 sin함수를 취하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  sin 노드는 입력된 값에 $cos(x)$를 곱하여 다음 노드로 전달합니다.

In [9]:
# sin 함수 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class Sin:
    
    # f(x) = sin(x)
    def forward(self, x):
        self.x = x
        # sin함수를 취하여 다음 노드로 전달합니다.
        return np.sin(self.x)
    
    # dfdx = cos(x)
    def backward(self):
        dx = np.cos(self.x)
        # cos(x)를 곱하여 다음 노드로 전달합니다.
        return dx

sin 노드 객체를 생성하고, **x=60&deg;**값을 넣어준 후 forward, backward값이 각각 **0.866, 0.5** 으로 잘 구해지는지 확인해 봅시다.
(numpy의 pi를 사용하여 x=np.pi / 3 로 넣어 줄 수 있습니다.)

In [10]:
# Sin 노드 객체를 생성합니다.
sin=Sin()
# x=60도 값을 대입해줍니다.
x=np.pi/3
# x를 입력받았을 때, sin노드의 forward, backward 값을 출력해줍니다. 
print(sin.forward(x), sin.backward())

0.8660254037844386 0.5000000000000001


#### 3-2. cos함수 노드 구현해보기
cos함수 노드의 수식이 $z=f(x)=cos(x)$일 때 $z$의 gradient는 아래와 같습니다.


$$
\nabla z = \left ( \frac{\partial z}{\partial x}\right ) = \left ( \frac{\partial cos(x)}{\partial x}\right ) = -sin(x)
$$

<img src="http://drive.google.com/uc?export=view&id=11AEEdSWOmBjGQhzW2-BDboXFjCvd-hi8" width="600">


>- **Forward Propagation**

>  cos 노드는 입력된 값에 cos함수를 취하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  cos 노드는 입력된 값에 $-sin(x)$를 곱하여 다음 노드로 전달합니다.

In [11]:
# cos 함수 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class Cos:
    
    # f(x) = cos(x)
    def forward(self, x):
        self.x = x
        # cos함수를 취하여 다음 노드로 전달합니다.
        return np.cos(self.x)
    
    # dfdx = -sin(x)
    def backward(self):
        dx = (-1) * np.sin(self.x)
        # -sin(x)를 곱하여 다음 노드로 전달합니다.
        return dx

cos 노드 객체를 생성하고, **x=60&deg;**값을 넣어준 후 forward, backward값이 각각 **0.5, -0.866** 으로 잘 구해지는지 확인해 봅시다.
(numpy의 pi를 사용하여 x=np.pi / 3 로 넣어 줄 수 있습니다.)

In [12]:
# cos 노드 객체를 생성합니다.
cos=Cos()
# x=60도 값을 대입해줍니다.
x=np.pi/3
# x를 입력받았을 때, cos노드의 forward, backward 값을 출력해줍니다. 
print(cos.forward(x), cos.backward())

0.5000000000000001 -0.8660254037844386


#### 3-3. tan함수 노드 구현해보기
tan함수 노드의 수식이 $z=f(x)=tan(x)$일 때 $z$의 gradient는 아래와 같습니다.


$$
\nabla z = \left ( \frac{\partial z}{\partial x}\right ) = \left ( \frac{\partial tan(x)}{\partial x}\right ) = \frac{1}{ cos(x)^2}
$$

<img src="http://drive.google.com/uc?export=view&id=16Q1LDQ4L2uY8dkqKgMRZP8UILcKtG10L" width="600">


>- **Forward Propagation**

>  tan 노드는 입력된 값에 tan함수를 취하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  tan 노드는 입력된 값에 $\frac{1}{ cos(x)^2}$를 곱하여 다음 노드로 전달합니다.

In [13]:
# tan 함수 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class Tan:
    
    # f(x) = tan(x)
    def forward(self, x):
        self.x = x
        # tan함수를 취하여 다음 노드로 전달합니다.
        return np.tan(self.x)
    
    # dfdx = 1/cos(x)^2
    def backward(self):
        dx = 1 /(np.cos(self.x)*np.cos(self.x))
        # 1/cos(x)^2를 곱하여 다음 노드로 전달합니다.
        return dx

cos 노드 객체를 생성하고, **x=60&deg;**값을 넣어준 후 forward, backward값이 각각 **1.732, 4** 으로 잘 구해지는지 확인해 봅시다.
(numpy의 pi를 사용하여 x=np.pi / 3 로 넣어 줄 수 있습니다.)

In [14]:
# tan 노드 객체를 생성합니다.
tan=Tan()
# x=60도 값을 대입해줍니다.
x=np.pi/3
# x를 입력받았을 때, tan노드의 forward, backward 값을 출력해줍니다. 
print(tan.forward(x), tan.backward())

1.7320508075688767 -3.9999999999999982


#### 이제, 그동안 다뤄봤었던 Loss Function을 Computational Graph로 그리고 이를 이용하여 Loss Function의 편미분을 구현해 보겠습니다. 

### 4. MSE Loss Function 노드 구현해보기

Regression 모델에서 주로 사용하는 평균제곱오차(MSE; Mean Squared Error) Loss Function (데이터 한 개의 Error)은 다음과 같이 나타낼 수 있습니다. (여기서는  $\frac{1}{2}$을 추가해주었는데 경우에 따라 생략하기도 합니다.)

$$
MSE=\frac{1}{2} (\hat{y}-y)^2
$$


데이터의 총 갯수를 n개라 하고, $y^{(i)}$는 i번째 데이터의 label값, $\hat{y}^{(i)}$은 ${y}^{(i)}$의 예측치로 $\hat{y}^{(i)}=\sigma(w^Tx^{(i)}+b)$라 하고 MSE의 Cost Function(비용함수:모든 Loss의 평균)를 나타내면 다음과 같습니다. 


$$
MSE(Cost)=\frac{1}{n}\sum_{i=1}^{n}\frac{1}{2} (\hat{y}^{(i)}-y^{(i)})^2
$$

Cost Function은 Loss Function의 단순 평균이므로, 여기서는 Loss Function만 구현해 보도록 하겠습니다.

위의 MSE Loss Function을 Computatioanl Graph로 그리면 다음과 같습니다. 

<img src="http://drive.google.com/uc?export=view&id=1dqfw6Tpeo6CkNsns8IMU3Bw1ItkAUNZ6" width="600">


이 Loss Function을 Gradient Descent 알고리즘에 적용하여 최적의 w와 b를 구해주기 위해서는 Loss 값을 w와 b로 편미분한 값을 필요로 했습니다. 따라서, 우리는 위 MSE Computational Graph에서 예측치인 $\hat{y}$에 대한 Loss의 편미분 값. $\frac{\partial L}{\partial \hat{y}} $ 을 구해주어야 합니다. 
즉, 지금 우리가 만들어주려하는 MSE 노드는 다음과 같이 나타낼 수 있습니다. 

<img src="http://drive.google.com/uc?export=view&id=16dbnhIahsvpVh1eAF4uavMA8mZIMkWhF" width="600">


이번 4번 과제는 위와 같이 MSE 노드에 $\hat{y}$과 $y$를 입력받았을 때, forward propagation으로 $ MSE=\frac{1}{2} (\hat{y}-y)^2 $ 을 출력하고, backward propagation으로는 $\frac{\partial L}{\partial \hat{y}} $ 값을 return 해주는 MSE 노드를 만드는 것입니다.

In [15]:
# MSE 노드에 사용할 Add노드를 만들어 줍니다. 
class Add:
    
    # f(x,y) = x + y
    def forward(self, x, y):
        self.x = x
        self.y = y
        # 두 가지 값을 더하여 다음 노드로 전달합니다.
        return self.x + self.y

    # dfdx = 1, dfdy = 1
    def backward(self):
        dx = 1
        dy = 1
        # 1을 곱하여 다음 노드로 전달합니다.
        return dx, dy

In [16]:
# MSE 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class MSE:
    
    # L = 1/2 * (y^-y)^2
    def forward(self, y_predict, y):
        # self.y_predict와 self.x에 입력받은 y_predict와 y값을 할당해줍니다. 
        self.y_predict = y_predict
        self.y = y
        #MSE 노드안에서 사용할 필요한 노드들을 생성해줍니다. 
        self.multiply1 = Multiply()
        self.add = Add()
        self.square = Square()
        self.multiply2 = Multiply()
        #forward propagation 과정을 적어줍니다. 
        forward1 = self.multiply1.forward(self.y, -1)
        forward2 = self.add.forward(forward1, self.y_predict)
        forward3 = self.square.forward(forward2)
        forward4 = self.multiply2.forward(forward3, 1/2)
        # 마지막 전달값인 forward4 값을 결과값으로 반환해 줍니다. 
        return forward4
    
    # dLd(y_predict) 
    def backward(self):
        #MSE Loss Function의 backward propagation 과정을 적어줍니다. 
        backward1 = self.multiply2.backward()[0]
        backward2 = backward1 * self.square.backward()
        backward3 = backward2 * self.add.backward()[0]
        
        # 마지막 전달값인 backward3 값을 결과값으로 반환해 줍니다. 
        return backward3

MSE 노드 객체를 생성하고, **y_predict = 1, y = 4**값을 넣어준 후 forward, backward값이 각각 **4.5, -3.0** 으로 잘 구해지는지 확인해 봅시다.

In [17]:
# MSE 노드 객체를 생성합니다.
mse=MSE()
# y_predict = 1, y = 4 값을 대입해줍니다.
y_predict = 1
y = 4
# y_predict 와 y를 입력받았을 때, mse노드의 forward, backward 값을 출력해줍니다. 
print(mse.forward(y_predict, y), mse.backward())

4.5 -3.0


### 5. Cross Entropy Loss Function 노드 구현해보기

Cross Entropy Loss Function은 다음과 같이 나타낼 수 있습니다.

class(label)의 총 갯수를 C개라 하고, $y_{c}$는 c 번째 label 값, $\hat{y}_{c}$은 ${y}_{c}$의 예측치로 $\hat{y}_{c}=\sigma(w^Tx_{c}+b)$라 하고 Cross Entropy Loss를 나타내면 다음과 같습니다. 

$$
\text{Cross Entropy Loss} = -\sum_{c=1}^{C}  y_{i} \times log(\hat{y}_{c})
$$

Binary Classification에서는 C값이 2이므로 다음과 같이 나타낼 수 있습니다.

(C값이 1이라고 생각하실 수 있지만, Cross Entropy Loss를 적용해줄 때는 원핫인코딩된 y를 사용하므로 Yes / No 두 개의 label이 생성됩니다. 그리고, $y_{1}+y_{2}=1$ 이므로 $y_{1}$을 그냥 $y$로 나타내면 아래와 같이 익숙한 Loss Function이 나옵니다. )

$$ \text{Cross Entropy Loss(binary)} = - y \times log(\hat{y}) - (1-{y}) \times log(1-\hat{y})$$

이번 과제에서는 Binary Classification이 아닌 Multi-Class, 즉, C=3인 Multi-Class Classification 의 Cross Entropy Loss를 Computational Graph로 그리고, 이 Cross Entropy Loss의 편미분을 구현해보도록 하겠습니다. 

C=3를 대입한 Cross Entropy Loss는 다음과 같고, 

$$
\text{Cross Entropy Loss} = -\sum_{c=1}^{3}  y_{c} \times log(\hat{y}_{c})
$$

이를 Computational Graph로 그려주면 다음과 같습니다. 

<img src="http://drive.google.com/uc?export=view&id=1HFLacOYnnN5ihTN9pm1_LEL1CB-1JNny" width="600">


이 Loss Function을 Gradient Descent 알고리즘에 적용하여 최적의 w와 b를 구해주기 위해서는 Loss 값을 w와 b로 편미분한 값을 필요로 했습니다. 따라서, 우리는 위 Cross Entropy의 Computational Graph에서 예측치인 $\hat{y}$에 대한 Loss의 편미분 값. $\frac{\partial L}{\partial \hat{y}} $ 을 구해주어야 합니다. 
즉, 지금 우리가 만들어주려하는 CE 노드는 다음과 같이 나타낼 수 있습니다. 

<img src="http://drive.google.com/uc?export=view&id=15wLtFHEr884R75PZrl7TLTUj2izJkWM-" width="600">


이번 5번 과제는 위와 같이 CE(Cross Entropy) 노드에 $\hat{y}$과 $y$를 입력받았을 때, forward propagation으로 $ \text{Cross Entropy Loss} = -\sum_{c=1}^{3}  y_{c} \times log(\hat{y}_{c})$ 을 출력하고, backward propagation으로는 $\frac{\partial L}{\partial \hat{y}} $ 값을 return 해주는 CE 노드를 만드는 것입니다.

In [18]:
# CE 노드에 사용할 Triple_Add노드를 만들어 줍니다. 
class Triple_Add:
    
    # f(x,y) = x + y
    def forward(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        # 세 가지 값을 더하여 다음 노드로 전달합니다.
        return self.x + self.y + self.z

    # dfdx = 1, dfdy = 1, dfdz = 1
    def backward(self):
        dx = 1
        dy = 1
        dz = 1
        # 1을 곱하여 다음 노드로 전달합니다.
        return dx, dy, dz

In [19]:
# CE 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class CE:
    
    # L = -y * log(y_predict)
    def forward(self, y_predict, y):
        # self.y_predict와 self.x에 입력받은 y_predict와 y값을 할당해줍니다. 
        self.y_predict = y_predict
        self.y = y
        #CE 노드안에서 사용할 필요한 노드들을 생성해줍니다. 
        self.log1 = Log()
        self.log2 = Log()
        self.log3 = Log()
        self.multiply1 = Multiply()
        self.multiply2 = Multiply()
        self.multiply3 = Multiply()
        self.triple_add = Triple_Add()
        self.multiply4 = Multiply()        

        #forward propagation 과정을 적어줍니다. 
        #먼저, log 부분입니다. 
        forward1 = self.log1.forward(self.y_predict[0])
        forward2 = self.log2.forward(self.y_predict[1])
        forward3 = self.log3.forward(self.y_predict[2])
        #다음으로 X 부분입니다. 
        forward4 = self.multiply1.forward(forward1, self.y[0])
        forward5 = self.multiply2.forward(forward2, self.y[1])  
        forward6 = self.multiply3.forward(forward3, self.y[2])
        # 덧셈노드(triple_add) 부분입니다. 
        forward7 = self.triple_add.forward(forward4, forward5, forward6)
        # 마지막 곱셈노드 입니다. 
        forward8 = self.multiply4.forward(forward7 , -1)
        # 마지막 전달값인 forward8 값을 결과값으로 반환해 줍니다. 
        return forward8
    
    # dLd(y_predict) 
    def backward(self):
        #CE Loss Function의 backward propagation 과정을 적어줍니다. 
        # 마지막 곱셈노드 부분입니다. 
        backward1 = self.multiply4.backward()[0]
        # 덧셈노드 부분입니다. 
        backward2 = backward1 * self.triple_add.backward()[0]
        backward3 = backward1 * self.triple_add.backward()[1]
        backward4 = backward1 * self.triple_add.backward()[2]
        # 3개의 곱셈노드 부분입니다. 
        backward5 = backward2 * self.multiply1.backward()[0]
        backward6 = backward3 * self.multiply2.backward()[0]
        backward7 = backward4 * self.multiply3.backward()[0]
        # log 부분입니다. 
        backward8 = backward5 * self.log1.backward()
        backward9 = backward6 * self.log2.backward()
        backward10 = backward7 * self.log3.backward()
        
        # 마지막 전달값인 backward8, backward9, backward10 값을 결과값으로 반환해 줍니다. 
        return backward8, backward9, backward10

CE 노드 객체를 생성하고, **y_predict_ce = [2, 3, 4] , y_ce = [0, 1, 2]** 값을 넣어준 후 forward, backward값이 각각 **-3.8712, (0, -0.33, -0.5)** 으로 잘 구해지는지 확인해 봅시다.

In [20]:
# CE 노드 객체를 생성합니다.
ce=CE()
# y_predict_ce = [2, 3, 4], y_ce = [0, 1, 2] 값을 대입해줍니다.
y_predict_ce = [2, 3, 4]
y_ce = [0, 1, 2]
# y_predict 와 y를 입력받았을 때, mse노드의 forward, backward 값을 출력해줍니다. 
print(ce.forward(y_predict_ce, y_ce), ce.backward())

-3.8712010109078907 (0.0, -0.3333333333333333, -0.5)


아래와 같이 pandas패키지를 이용하면 log, multiply 노드를 한번에 해결해 줄 수 있습니다. (add노드의 backward는 1을 곱하는 것이므로 생략)

위의 코드보다 비교적 간단하게 쓸 수 있지만, pandas 패키지의 느린 연산속도 때문에 많은 양의 데이터를 사용하는 실제 현장에서는 잘 사용하지 않습니다.

In [21]:
y_predict_ce = [2,3,4]
y_ce = [0,1,2]

In [22]:
import pandas as pd
y_predict_ce = pd.Series(y_predict_ce)
y_ce = pd.Series(y_ce)

In [23]:
# CE 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class CE2:
    
    # L = -y * log(y_predict)
    def forward(self, y_predict, y):
        # self.y_predict와 self.x에 입력받은 y_predict와 y값을 할당해줍니다. 
        self.y_predict = y_predict
        self.y = y
        #CE 노드안에서 사용할 필요한 노드들을 생성해줍니다. 
        self.log = Log()
        self.mul = Multiply()
        self.mul2 = Multiply() 

        #forward propagation 과정을 적어줍니다. 
        forward1 = self.log.forward(self.y_predict)
        forward2 = self.mul.forward(forward1, self.y)
        forward3 = forward2.sum()
        forward4 = self.mul2.forward(forward3, -1)
        
        # 마지막 전달값인 forward4 값을 결과값으로 반환해 줍니다. 
        return forward4
    
    # dLd(y_predict) 
    def backward(self):
        #CE Loss Function의 backward propagation 과정을 적어줍니다. 
        backward1 = self.mul2.backward()[0]
        backward2 = backward1 * self.mul.backward()[0]
        backward3 = backward2 * self.log.backward()
        
        # 마지막 전달값인 backward3 값을 결과값으로 반환해 줍니다. 
        return backward3

In [24]:
# CE 노드 객체를 생성합니다.
ce2=CE2()
# y_predict_ce = [2, 3, 4], y_ce = [0, 1, 2] 값을 대입해줍니다.
y_predict_ce = [2, 3, 4]
y_ce = [0, 1, 2]
# 리스트를 pandas의 Series로 바꿔줍니다. 
import pandas as pd
y_predict_ce = pd.Series(y_predict_ce)
y_ce = pd.Series(y_ce)

# y_predict 와 y를 입력받았을 때, mse노드의 forward, backward 값을 출력해줍니다. 
print(ce2.forward(y_predict_ce, y_ce))
print(ce2.backward())

-3.8712010109078907
0    0.000000
1   -0.333333
2   -0.500000
dtype: float64


## 제출

과제를 다 끝내셨으면 http://bit.ly/ds-assignment 에서 안내에 따라 과제를 제출하여 주세요! 과제를 제출해주시면 검토 결과를 드리고 다음 레벨에 초대해드립니다. 

과제 제출 후에 늦어도 24시간 이내에는 회신을 드립니다. 다만 과제의 난이도나 풀이에 따라 다소 시간이 걸릴 수 있다는 점 양해 부탁드립니다. 혹시나 다음 레벨로 초대를 못 받은 분들은 채널에 있는 튜터분에게 문의 주시면 다음 채널을 오픈해드립니다.

수업이나 과제 관련 질문, 수료증 문의 등은 담당 튜터(조교)에게 문의 주세요. 영수증 발급 등의 문의는 support@dsschool.co.kr 로 메일 주시면 담당자분이 응대해주실 겁니다. 기타 궁금한 사항은 슬랙으로 문의 주세요!